In [42]:
# General Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Web Scraping Libraries
import urllib
import requests
from bs4 import BeautifulSoup

# Regex Library
import re

# Time-related Libraries
import time
import datetime

# NLP Libraries
import unicodedata
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from wordcloud import WordCloud

# Helper functions
import MVP_Bojado, MVP_Shi

# Environment file
import env, env_Shi

# AWS
import logging
import boto3
from botocore.exceptions import ClientError

# Geospatial Libraries
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import folium


import json
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings("ignore")

## Data Acquisition

<div class="alert alert-block alert-success"><b>All the functions in the Data Acquisitioin section have been tested out and inorporated into the MVP_acquire_ds.py and MVP_acquire_wd.py files. To save space, no extra test is carried out in this notebook.</b></div>

### URL Format of Indeed.com
1. Search chemist in TX<br>
https://www.indeed.com/jobs?q=chemist&l=TX
2. Search chemist in San Antonio, TX<br>
https://www.indeed.com/jobs?q=chemist&l=San+Antonio%2C+TX
3. Search data scientist in San Antonio, TX<br>
https://www.indeed.com/jobs?q=data+scientist&l=San+Antonio%2C+TX
4. Search data scientist intern in San Anotnio, TX<br>
https://www.indeed.com/jobs?q=data+scientist+intern&l=San+Antonio%2C+TX
5. Sort the data scientist jobs posting by date<br>
https://www.indeed.com/jobs?q=data+scientist&l=San+Antonio%2C+TX&sort=date

**Takeaways**
1. q = job title
2. l = location

### URL Format of Monster.com
https://www.monster.com/jobs/search/?q=data-scientist&where=San-Antonio__2C-TX

### Generate the URL of a Job Search at Indeed.com

In [ ]:
def first_page_url_indeed(job_title, location):
    '''
    This function returns a URL of the 1st page of a job search at Indeed.com 
    based on the job title and the location.
    '''
    # Create the base URL for a job serch at Indeed.com
    base_url = 'https://www.indeed.com/jobs?'
    # Create a dictionary to map the keys to the input parameters
    dic = {'q': job_title, 'l': location, 'sort': 'date'}
    # Convert the dictionary to a query string
    relative_url = urllib.parse.urlencode(dic)
    # Generate the full URL of the first page
    url = base_url + relative_url
    return url

### Make the HTTP Request

In [ ]:
def first_page_soup_indeed(job_title, location):
    '''
    This function returns a BeautifulSoup object to hold the content 
    of the first page of a request for job searching at Indeed.com
    '''
    # Generate the URL of the job search based on title and location
    url = first_page_url_indeed(job_title, location)
    # Make the HTTP request
    response = requests.get(url)
    # Print the status code of the request
    print("Status code of the request: ", response.status_code)
    # Sanity check to make sure the document type is HTML
    print("Document type: ", response.text[:15])
    # Take a break
    time.sleep(5)
    # Make a soup to hold the response content
    soup = BeautifulSoup(response.content, "html.parser")
    # Print out the title of the content
    print("Title of the response: ", soup.title.string)
    return soup

In [ ]:
# first_page_soup = first_page_soup_indeed("data scientist", 'al')
# type(first_page_soup)

# # Find out the tag that contains the number of the jobs by seaching

# num_jobs = first_page_soup.find('div', id='searchCountPages')
# print("Data Type: ", type(num_jobs))
# print("Name of the Tag: ", num_jobs.name)
# print("Attributes of the Tag: ", num_jobs.attrs)
# print("Text within the Tag: ")
# num_jobs.text

# # Find the number of the jobs in the text
# match = re.findall(r'(\d+)', num_jobs.text)
# match[1]

In [ ]:
def num_jobs_indeed(first_page_soup):
    '''
    This function returns the total number of the jobs in the searching result.
    '''
    # Find out the section contains total number of jobs  
    div = first_page_soup.find('div', id='searchCountPages')
    # Extract the number
    num_jobs = re.findall(r'(\d+)', div.text)[1]
    return num_jobs

In [ ]:
def page_num_indeed(url):
    '''
    This function returns the page number of job searching results. 
    '''
    # Create a Soup object based on the url
    soup = page_soup_indeed(url)
    # Find out the section contains total number of jobs  
    div = soup.find('div', id='searchCountPages')
    # Extract the number
    page_num = re.findall(r'(\d+)', div.text)[0]
    return page_num

In [ ]:
# Define a function to extract all job cards in a Indeed page

def job_cards_indeed(soup):
    '''
    This function accepts the Soup object of a Indeed page 
    return an iterator containing the all the job cards in this page.
    '''
    # Find the appropriate tag that contains all of the job listings in this page
    tag = soup.find('td', id="resultsCol")
    # Extract all job cards
    job_cards = tag.find_all('div', class_='jobsearch-SerpJobCard')
    return job_cards

In [ ]:
# # Test the function job_cards_indeed
# job_cards = job_cards_indeed(first_page_soup)

# # Print the data type of job_cards
# type(job_cards)

# # How many jobs listed in the 1st page? 
# len(job_cards)

In [ ]:
def job_titles_indeed(job_cards):
    '''
    This function extract the job titles from a job_cards set. 
    '''
    # Create a list to hold the job titles
    titles = []
    # For Loop throught the job cards to extract the titles
    for job in job_cards:
        title = job.find('h2', class_='title')
        title = title.text.strip()
        titles.append(title)
    return titles

In [ ]:
# Define a function to pull the company names from a set of job cards

def company_names_indeed(job_cards):
    '''
    This function extracts the company names from a set of job cards.
    '''
    # Create a list to hold the company names
    names = []
    # For loop through the job cards to pull the company names
    for job in job_cards:
        name = job.find('span', class_='company')
        name = name.text.strip()
        names.append(name)
    return names

In [ ]:
# Define a function to pull the post ages from a set of job cards

def post_ages_indeed(job_cards):
    '''
    This function pulls the post ages from a set of job cards.
    '''
    # Create a list to hold the post ages
    ages = []
    # For loop through the job cards to pull the post ages
    for job in job_cards:
        age = job.find('span', class_='date')
        age = age.text.strip()
        ages.append(age)
    return ages

In [ ]:
# Define a function to pull the location from a set of job cards

def job_locations_indeed(job_cards):
    '''
    This function pulls the job locations from a set of job cards.
    '''
    # Create a list to hold the locations
    locations = []
    # For loop through the job cards to pull the locations
    for job in job_cards:
        location = job.find('div', class_='location accessible-contrast-color-location')
        if location == None:
            location = job.find('span', class_='location accessible-contrast-color-location')
        location = location.text.strip()
        locations.append(location)
    return locations

In [ ]:
# Define a function to pull the company ratings from a set of job cards

def company_rating_indeed(job_cards):
    '''
    This function pulls the company rating from a set of job cards.
    If the rating is unavailable, it will be marked as 'missing'.
    '''
    # Create a list to hold the locations
    ratings = []
    # For loop through the job cards to pull the locations
    for job in job_cards:
        rating = job.find('span', class_='ratingsContent')
        if rating == None:
            ratings.append('missing')
            continue
        rating = rating.text.strip()
        ratings.append(rating)
    return ratings

In [ ]:
def acuqire_indeed_job_description(url):
    '''
    This function accepts the URL of a job posting and pull its description.
    '''
    # Make the HTTP request
    request = requests.get(url)
    print("Status Code: ", request.status_code)
    # Take a break
    time.sleep(5)
    # Make a soup variable holding the response content
    soup = BeautifulSoup(request.content, "html.parser")
    if soup == None:
        description = 'error'
    else:
        # Print the page's title
        print(soup.title.string)
        # Find the section that contains job description
        description = soup.find('div', id="jobDescriptionText")
        if description == None:
            description = 'error'
        else:
            description = description.text
    return description

def job_links_and_contents_indeed(job_cards):
    '''
    This function pulls the job links and descriptions from a set of job cards.
    '''
    # Create a list to hold the links and descriptions
    links = []
    descriptions = []
    # For loop through the job cards to pull the links and descriptions
    for job in job_cards:
        link = job.find('a')['href']
        link = 'https://www.indeed.com' + link
        link = link.replace(';', '&')
        description = acuqire_indeed_job_description(link)
        links.append(link)
        descriptions.append(description)
    return links, descriptions

In [ ]:
# Define a function to create a Soup object based on a job search url

def page_soup_indeed(url):
    '''
    This function returns a BeautifulSoup object to hold the content 
    of a page for a job searching results at Indeed.com
    '''
    # Make the HTTP request
    response = requests.get(url)
    # Print the status code of the request
    print("Status code of the request: ", response.status_code)
    # Sanity check to make sure the document type is HTML
    print("Document type: ", response.text[:15])
    # Take a break
    time.sleep(5)
    # Make a soup to hold the response content
    soup = BeautifulSoup(response.content, "html.parser")
    # Print out the title of the content
    print("Title of the response: ", soup.title.string)
    return soup

In [ ]:
# # Test the function: page_soup_indeed

# url = 'https://www.indeed.com/jobs?q=data+scientist&l=al&sort=date'
# soup = page_soup_indeed(url)
# type(soup)

# # Find out the page number
# int(page_num_indeed(url))

# # Pull the job cards from the soup
# type(job_cards_indeed(soup))

In [ ]:
# Define a function to pull job information from a job search URL

def acquire_page_indeed(url):
    '''
    This function accepts a job search URL and returns a pandas dataframe 
    containing job title, location, company, company rating, post age and description. 
    '''
    # Create a Soup object based on the url
    soup = page_soup_indeed(url)
    # Pull the job cards
    job_cards = job_cards_indeed(soup)
    # Pull the job titles
    titles = job_titles_indeed(job_cards)   
    # Pull the names of the companies
    companies = company_names_indeed(job_cards)
    # Pull the post ages
    ages = post_ages_indeed(job_cards)
    # Pull the job locations
    locations = job_locations_indeed(job_cards)
    # Pull the company ratings
    ratings = company_rating_indeed(job_cards)
    # Pull the hyperlinks and job description
    links, descriptions = job_links_and_contents_indeed(job_cards)    
    # Create a dataframe
    d = {'title': titles,
         'location': locations,
         'company': companies, 
         'company_rating': ratings,
         'post_age': ages, 
         'job_link': links, 
         'job_description': descriptions}
    df = pd.DataFrame(d)
    return df

In [ ]:
def jobs_indeed(job_title, location, max_page=35):
    '''
    This function accepts the job title and location and return the job information (35 pages by default) 
    pulled from Indeed.com.
    '''
    # Generate the urls based on job title and location (state)
    url = first_page_url = first_page_url_indeed(job_title, location)
    # Set up an counter
    counter = 1
    # Create an empty dataframe to hold the job information
    df_jobs = pd.DataFrame(columns = ['title', 'location', 'company', 'company_rating', 
                                      'post_age','job_link', 'job_description'])
    # Pull the page number
    page_num = int(page_num_indeed(url))
    # Set up an checker
    keep_going = (counter == page_num)   
    # For loop through the urls to pull job information
    while keep_going and page_num <= max_page:
        df = acquire_page_indeed(url)
        print("--------------------------------")
        print("Page: ", page_num)
        print("--------------------------------")
        df_jobs = df_jobs.append(df, ignore_index=True)
        df_jobs.to_csv("df_jobs_backup.csv")
        time.sleep(180)
        dic = {'start': page_num*10}
        relative_url = urllib.parse.urlencode(dic)
        url = first_page_url + '&' + relative_url
        counter = counter + 1
        page_num = int(page_num_indeed(url))
        keep_going = (counter == page_num)
    # Print the total number of jobs
    print(f"Total number of {job_title} positions in {location}: ", df_jobs.shape[0])
    return df_jobs

## Data Preparation

In [55]:
# Define a function to remove the duplicates

def remove_duplicates(df):
    '''
    This function removes the duplicates in the dataframe
    '''
    # Define the columns for identifying duplicates
    columns = ['title', 'location', 'company', 'job_link', 'job_description']
    # Drop the duplicates except for the last occurrence
    df.drop_duplicates(subset=columns, inplace=True, keep='last')
    return df

In [56]:
# Define a function to compute the date of the job posts

def compute_post_date(df):
    '''
    This function computes the date of the job post based on post age
    and set the date as the index of the dataframe.
    '''
    # Create an empty list to hold the post date
    post_date = []
    # For loop the column post_age and convert the values to date
    for age in df.post_age:
        if age == 'Just posted':
            date = datetime.date.today()
            post_date.append(date)
        elif age == 'Today':
            date = datetime.date.today()
            post_date.append(date)
        else:
            # Extract the number
            num = re.findall(r'(\d+)', age)[0]
            # Cast the string number to integer
            num = int(num)
            # Convert the integer to timedelta object
            num = datetime.timedelta(days=num)
            # Compute post date        
            date = datetime.date.today()
            date = date - num
            post_date.append(date)
    # Add post date as new column
    df['date'] = post_date
    # Set the column post_date as the index and sort the values
    df = df.set_index('date').sort_index(ascending=False)
    return df

In [63]:
# Define a function to clean the job titles for analysis

def clean_job_title(title):
    '''
    This function removes the "\nnew" and "..." in the job title.
    '''
    title = title.split(sep="\nnew")[0]
#     title = title.split(sep=' -')[0]
#     title = title.split(sep=' (')[0]
#     title = title.split(sep=',')[0]
    title = title.split(sep='...')[0]
    return title

In [23]:
# Define a function to transform old job posts files

def transform_old_file(df, date_string):
    '''
    This function accepts old daily job posts and convert the post age to post date. 
    '''
    # Create an empty list to hold the post date
    post_date = []
    # For loop the column post_age and convert the values to date
    for age in df.post_age:
        if age == 'Just posted':
            date = datetime.date.fromisoformat(date_string)
            post_date.append(date)
        elif age == 'Today':
            date = datetime.date.fromisoformat(date_string)
            post_date.append(date)
        else:
            # Extract the number
            num = re.findall(r'(\d+)', age)[0]
            # Cast the string number to integer
            num = int(num)
            # Convert the integer to timedelta object
            num = datetime.timedelta(days=num)
            # Compute post date        
            date = datetime.date.fromisoformat(date_string)
            date = date - num
            post_date.append(date)
    # Add post date as new column
    df['date'] = post_date
    # Set the column post_date as the index and sort the values
    df = df.set_index('date').sort_index(ascending=False)
    return df

### Web Deveopment

In [51]:
# When is the df_wd_tx_prepared.json last modified?

s3 = boto3.resource('s3')
object_json = s3.Object('wdpreparedjobpostings', 'df_wd_tx_prepared_backup.json')
object_json.last_modified

datetime.datetime(2021, 2, 20, 3, 21, 30, tzinfo=tzutc())

In [38]:
# # Load job posts of web developer in TX on Feb.12 2021

# # Import the file path
# database = env_Shi.database

# # Read the daily data scientist jobs in TX
# df_wd_old = pd.read_csv(f"{database}web_developer_tx_indeed_021221.csv", index_col=0)

# # Print the first 2 rows
# df_wd_old.head(2)

# # Transform old file
# df_wd_old = transform_old_file(df_wd_old, '2021-02-12')
# df_wd_old.head(2)

In [5]:
# Load web developer job posts in TX on Feb 19 2021

# Import the file path
database = env_Shi.database

# Read the daily data scientist jobs in TX
df_wd_new = pd.read_csv(f"{database}web_developer_tx_indeed_021921.csv", index_col=0)

# Print the dimentionality
print(df_wd_new.shape)

# Print the first two rows
df_wd_new.head(2)

(300, 7)


,title,location,company,company_rating,post_age,job_link,job_description
0,Sr. PHP Developer\nnew,United States,The Lead Group,missing,Just posted,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,We are a rapidly growing Online Lead Generatio...
1,Sales Lead Generation for Web Development Busi...,"San Antonio, TX",Buddy Web Design & Development,missing,Just posted,https://www.indeed.com/company/Buddy-Web-Desig...,Buddy Web Design & Development is a young and ...


In [6]:
def daily_update_wd(df):
    '''
    This function updates job posts of web developer in TX by adding the daily acquring
    of web developer job posts in TX. 
    '''
    # Read the job posts of web developer in TX
    database = env_Shi.database
    df_wd_tx = pd.read_csv(f"{database}df_wd_tx_backup.csv")
    num_jobs = df_wd_tx.shape[0]
    # Convert the date column to datetime type
    df_wd_tx.date = pd.to_datetime(df_wd_tx.date)
    # Set the date column as the index and sort the index
    df_wd_tx = df_wd_tx.set_index('date').sort_index(ascending=False)
    # Add the daily update
    df = compute_post_date(df)
    df_wd_tx = pd.concat([df_wd_tx, df]).sort_index(ascending=False)
    # Remove the duplicates
    df_wd_tx = remove_duplicates(df_wd_tx)
    # Save as csv file
    df_wd_tx.to_csv(f"{database}df_wd_tx_backup.csv")
    num_new_jobs = df_wd_tx.shape[0] - num_jobs
    print("New Jobs Posted Today: ", num_new_jobs)
    return df_wd_tx

In [7]:
# Test function: daily_update_wd

df_test = daily_update_wd(df_wd_new)
df_test.head(2)

New Jobs Posted Today:  96


,title,location,company,company_rating,post_age,job_link,job_description
date,,,,,,,
2021-02-19,Sr. PHP Developer\nnew,United States,The Lead Group,missing,Just posted,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,We are a rapidly growing Online Lead Generatio...
2021-02-19,Web Designer\nnew,"Dallas, TX 75218 (Northeast Dallas area)",The Old State,missing,Today,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,We are looking for an experienced Web/Graphic ...


In [8]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3965 entries, 2021-02-19 to 2021-01-04 00:00:00
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            3965 non-null   object
 1   location         3965 non-null   object
 2   company          3965 non-null   object
 3   company_rating   3965 non-null   object
 4   post_age         3965 non-null   object
 5   job_link         3965 non-null   object
 6   job_description  3965 non-null   object
dtypes: object(7)
memory usage: 247.8+ KB


In [9]:
# Define a function to prepare the job posts of web developer

def prepare_job_posts_indeed_wd():
    '''
   The function cleans the csv file of web developer job posts and save as json. 
    '''
    # Read the job posts of web developer in TX
    database = env_Shi.database
    df = pd.read_csv(f"{database}df_wd_tx_backup.csv")
    # Create columns of city, state, and zipcode
    location = df.location.str.split(', ', expand=True)
    location.columns = ['city', 'zipcode']
    location.city = location.city.apply(lambda i: 0 if i == 'United States' else i)
    location.city = location.city.apply(lambda i: 0 if i == 'Texas' else i)
    location.zipcode = location.zipcode.apply(lambda i: 0 if re.findall(r"(\d+)", str(i)) == [] 
                                          else re.findall(r"(\d+)", str(i))[0])
    df['city'] = location.city
    df['state'] = 'TX'
    df['zipcode'] = location.zipcode
    # Replace the missing values in the company rating with 0
    df.company_rating = df.company_rating.apply(lambda i: 0 if i == 'missing' else i)
    # Drop the column post_age and location
    df = df.drop(columns=['post_age', 'location'])
    # Clean the text in the job description
    df = MVP_Bojado.prep_job_description_data(df, 'job_description')
    # Clean the job title
    df.title = df.title.apply(clean_job_title)
    # Save a JSON version of the prepared data
    df.to_json(f"{database}df_wd_tx_prepared_backup.json", orient='records')
    return df

In [10]:
%%time

# Test the function: prepare_job_posts_indeed_wd
df_test = prepare_job_posts_indeed_wd()
df_test.head(2)

CPU times: user 46.7 s, sys: 471 ms, total: 47.2 s
Wall time: 47.4 s


,date,title,company,company_rating,job_link,job_description,city,state,zipcode,clean,tokenized,stemmed,lemmatized
0,2021-02-19,Sr. PHP Developer\nnew,The Lead Group,0,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,We are a rapidly growing Online Lead Generatio...,0,TX,0,rapidly growing online lead generation company...,we are a rapidly growing online lead generatio...,we are a rapidli grow onlin lead gener compani...,we are a rapidly growing online lead generatio...
1,2021-02-19,Web Designer\nnew,The Old State,0,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,We are looking for an experienced Web/Graphic ...,Dallas,TX,75218,looking experienced webgraphic designer join c...,we are looking for an experienced webgraphic d...,we are look for an experienc webgraph design t...,we are looking for an experienced webgraphic d...


In [11]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3965 entries, 0 to 3964
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   date             3965 non-null   object
 1   title            3965 non-null   object
 2   company          3965 non-null   object
 3   company_rating   3965 non-null   object
 4   job_link         3965 non-null   object
 5   job_description  3965 non-null   object
 6   city             3965 non-null   object
 7   state            3965 non-null   object
 8   zipcode          3965 non-null   object
 9   clean            3965 non-null   object
 10  tokenized        3965 non-null   object
 11  stemmed          3965 non-null   object
 12  lemmatized       3965 non-null   object
dtypes: object(13)
memory usage: 402.8+ KB


In [221]:
# Adjust the data types

dtypes = {'company_rating': 'float16', 
          'zipcode': 'int16'}
df_test.astype(dtypes).dtypes

date                object
title               object
company             object
company_rating     float16
job_link            object
job_description     object
city                object
state               object
zipcode              int16
clean               object
tokenized           object
stemmed             object
lemmatized          object
dtype: object

In [12]:
# Define the columns for identifying duplicates
columns = ['date', 'title', 'company', 'job_link', 'job_description', 'city', 'state', 'zipcode']
   
# Check for duplicates
duplicates = df_test.duplicated(subset=columns,keep='last')
duplicates.sum()

0

In [ ]:
# # Read the json file

# result = open(f"{database}df_wd_tx_prepared_backup.json")
# parsed = json.load(result)
# parsed[1]

In [13]:
%%time
# Upload the json file to AWS

# Create the s3 resource object
s3 = boto3.resource('s3')

# Read the location of the database
database = env_Shi.database

# Upload df_ds_tx_backup.csv file
s3.Bucket('wdrawjobpostings').upload_file(f"{database}df_wd_tx_backup.csv", "df_wd_tx_backup.csv")

# Upload df_ds_tx_prepared_backup.json file
s3.Bucket('wdpreparedjobpostings').upload_file(f"{database}df_wd_tx_prepared_backup.json", 
                                               "df_wd_tx_prepared_backup.json")

CPU times: user 1.85 s, sys: 1.49 s, total: 3.34 s
Wall time: 1min 16s


### Data Scientist

In [82]:
# Print the time when the prepared json file is last modified

s3 = boto3.resource("s3")
prepared_json = s3.Object('dspreparedjobpostings', 'df_ds_tx_prepared_backup.json')
prepared_json.last_modified

datetime.datetime(2021, 2, 21, 17, 29, 59, tzinfo=tzutc())

In [ ]:
# # Load old data scientist job posts in TX

# # Import the file path
# database = env_Shi.database

# # Read the daily data scientist jobs in TX
# df_ds_old = pd.read_csv(f"{database}data_scientist_tx_indeed_020821.csv", index_col=0)

# # Print the first 2 rows
# df_ds_old.head(2)

# # Transform old file
# df_test = transform_old_file(df_ds_old, '2021-02-08')
# df_test.head(2)

In [53]:
# Load data scientist job posts in TX on 2021-02-21

# Import the file path
database = env_Shi.database

# Read the daily data scientist jobs in TX
df_ds_new = pd.read_csv(f"{database}data_scientist_tx_indeed_022121.csv", index_col=0)

# Inspect the first 2 rows of the new posts
df_ds_new.head(2)

,title,location,company,company_rating,post_age,job_link,job_description
0,Advanced Analytics Technical Project Manager (...,"Irving, TX 75061",NTT DATA Corporation,3.5,Just posted,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Req ID: 113068\nNTT DATA Services strives to h...
1,Vice President of Decision Science\nnew,"Carrollton, TX 75006",Crescent Bank,3.3,Today,https://www.indeed.com/rc/clk?jk=434aacd861bb9...,Vice President of Decision Science\nCarrollton...


In [57]:
# Test the function: compute_post_date

df_test = compute_post_date(df_ds_new)
df_test.head(2) # Works

,title,location,company,company_rating,post_age,job_link,job_description
date,,,,,,,
2021-02-21,Advanced Analytics Technical Project Manager (...,"Irving, TX 75061",NTT DATA Corporation,3.5,Just posted,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Req ID: 113068\nNTT DATA Services strives to h...
2021-02-21,Vice President of Decision Science\nnew,"Carrollton, TX 75006",CRESCENT BANK & TRUST,3.3,Today,https://www.indeed.com/cmp/Crescent-Bank,error


In [58]:
def daily_update_ds(df):
    '''
    This function updates job posts of data scientist in TX by adding the daily acquring
    of data scientist job posts in TX. 
    '''
    # Read the job posts of data scientist in TX
    database = env_Shi.database
    df_ds_tx = pd.read_csv(f"{database}df_ds_tx_backup.csv")
    num_jobs = df_ds_tx.shape[0]
    # Convert the date column to datetime type
    df_ds_tx.date = pd.to_datetime(df_ds_tx.date)
    # Set the date column as the index and sort the index
    df_ds_tx = df_ds_tx.set_index('date').sort_index(ascending=False)
    # Add the daily update
    df = compute_post_date(df)
    df_ds_tx = pd.concat([df_ds_tx, df]).sort_index(ascending=False)
    # Remove the duplicates
    df_ds_tx = remove_duplicates(df_ds_tx)
    # Save as csv file
    df_ds_tx.to_csv(f"{database}df_ds_tx_backup.csv")
    # Print the new jobs posted today
    num_new_jobs = df_ds_tx.shape[0] - num_jobs
    print("New Jobs Posted Today: ", num_new_jobs)
    return df_ds_tx

In [59]:
# Test the function: daily_update_ds

df_test = daily_update_ds(df_ds_new)
df_test.head() # Works

New Jobs Posted Today:  69


,title,location,company,company_rating,post_age,job_link,job_description
date,,,,,,,
2021-02-21,Vice President of Decision Science\nnew,"Carrollton, TX 75006",CRESCENT BANK & TRUST,3.3,Today,https://www.indeed.com/cmp/Crescent-Bank,error
2021-02-21,"Data Scientist, Analytics - Ads Delivery Produ...","Austin, TX (Downtown area)",Facebook,4.2,Today,https://www.indeed.com/rc/clk?jk=3ec2fc58833a3...,The Ads Delivery Product team leverages state-...
2021-02-21,Data Visualization Engineer\nnew,"Dallas, TX",TA Digital,missing,Today,https://www.indeed.com/rc/clk?jk=92f9610265697...,Location: Anywhere in the US | Location Prefer...
2021-02-21,Senior Forecasting & Decision Scientist\nnew,"Plano, TX 75023",Toyota,4.0,Today,https://www.indeed.com/rc/clk?jk=c35d88a8c4ffd...,Who We Are\nCollaborative. Respectful. A place...
2021-02-21,Vice President of Decision Science\nnew,"Carrollton, TX 75006",Crescent Bank,3.3,Today,https://www.indeed.com/rc/clk?jk=434aacd861bb9...,Vice President of Decision Science\nCarrollton...


In [60]:
# Print the information of the dateframe
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2136 entries, 2021-02-21 to 2020-12-22 00:00:00
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            2136 non-null   object
 1   location         2136 non-null   object
 2   company          2136 non-null   object
 3   company_rating   2136 non-null   object
 4   post_age         2136 non-null   object
 5   job_link         2136 non-null   object
 6   job_description  2136 non-null   object
dtypes: object(7)
memory usage: 133.5+ KB


In [68]:
# Define a function to prepare the job post for exploration

def prepare_job_posts_indeed_ds():
    '''
    The function cleans the csv file of data scientist job posts and save as json. 
    '''
    # Read the job posts of data scientist in TX
    database = env_Shi.database
    df = pd.read_csv(f"{database}df_ds_tx_backup.csv")
    # Create columns of city, state, and zipcode
    location = df.location.str.split(', ', expand=True)
    location.columns = ['city', 'zipcode']
    location.city = location.city.apply(lambda i: 0 if i == 'United States' else i)
    location.city = location.city.apply(lambda i: 0 if i == 'Texas' else i)
    location.zipcode = location.zipcode.apply(lambda i: 0 if re.findall(r"(\d+)", str(i)) == [] 
                                          else re.findall(r"(\d+)", str(i))[0])
    df['city'] = location.city
    df['state'] = 'TX'
    df['zipcode'] = location.zipcode
    # Replace the missing values in the company rating with 0
    df.company_rating = df.company_rating.apply(lambda i: 0 if i == 'missing' else i)
    # Clean the text in the job description
    df = MVP_Bojado.prep_job_description_data(df, 'job_description')
    # Clean the job title
    df.title = df.title.apply(clean_job_title)
    # Drop the redundant columns post_age and location
    redundant_cols = ['post_age', 'location', 'tokenized', 'stemmed', 'lemmatized']
    df = df.drop(columns=redundant_cols)
    # Save a JSON version of the prepared data
    df.to_json(f"{database}df_ds_tx_prepared_backup.json", orient='records')
    return df

In [69]:
%%time
# Test the function: prepare_job_posts_indeed

df_test = prepare_job_posts_indeed_ds()
df_test.head(2)

CPU times: user 32.4 s, sys: 251 ms, total: 32.6 s
Wall time: 32.9 s


,date,title,company,company_rating,job_link,job_description,city,state,zipcode,clean
0,2021-02-21,Vice President of Decision Science,CRESCENT BANK & TRUST,3.3,https://www.indeed.com/cmp/Crescent-Bank,error,Carrollton,TX,75006,error
1,2021-02-21,"Data Scientist, Analytics - Ads Delivery Product",Facebook,4.2,https://www.indeed.com/rc/clk?jk=3ec2fc58833a3...,The Ads Delivery Product team leverages state-...,Austin,TX,0,ad delivery product team leverage stateofthear...


In [71]:
# Print the information of the dataframe
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2136 entries, 0 to 2135
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   date             2136 non-null   object
 1   title            2136 non-null   object
 2   company          2136 non-null   object
 3   company_rating   2136 non-null   object
 4   job_link         2136 non-null   object
 5   job_description  2136 non-null   object
 6   city             2136 non-null   object
 7   state            2136 non-null   object
 8   zipcode          2136 non-null   object
 9   clean            2136 non-null   object
dtypes: object(10)
memory usage: 167.0+ KB


In [72]:
# Define the columns for identifying duplicates
columns = ['date', 'title', 'company', 'job_link', 'job_description', 'city', 'state', 'zipcode']
   
# Check for duplicates
duplicates = df_test.duplicated(subset=columns,keep='last')
duplicates.sum()

207

<div class="alert alert-block alert-info"><b>Takeaways:</b>
1. After the job titles are cleaned, the duplicates of job postings of data scientist positions increase from 1 to 207. It suggests that when the same job is re-posted, the job title changes. 
</div>

### Upload Files to AWS S3 Bucket
- Up-to-date job postings of data scientist positions in TX
- Cleaned job postings of data scientist positions in TX

In [81]:
# Create the s3 resource object
s3 = boto3.resource('s3')

# Read the location of the database
database = env_Shi.database

# Upload df_ds_tx_backup.csv file
s3.Bucket('dsrawjobpostings').upload_file(f"{database}df_ds_tx_backup.csv", "df_ds_tx_backup.csv")

# Upload df_ds_tx_prepared_backup.json file
s3.Bucket('dspreparedjobpostings').upload_file(f"{database}df_ds_tx_prepared_backup.json", 
                                               "df_ds_tx_prepared_backup.json")

**Notes**: The two files have been successfully uploaded to the designated bucket. 

# Data Exploration

### Downlaod JSON Files from AWS S3 Bucket

In [2]:
# Create the s3 resource object
s3 = boto3.resource('s3')

# Print the data type of s3
print(type(s3))

# Print the bucket names
for bucket in s3.buckets.all():
    print(bucket.name)

<class 'boto3.resources.factory.s3.ServiceResource'>
additionaljobinfo
amplify-jobdashboardfront-dev-180611-deployment
dspreparedjobpostings
dsrawjobpostings
sagemaker-jobsearchdashboard
sagemaker-studio-793555146825-q0wiagiqq8r
wdpreparedjobpostings
wdrawjobpostings


In [3]:
# Create the bucket object
ds_raw = s3.Bucket('dsrawjobpostings')

# Print the data type of ds_raw
type(ds_raw)

boto3.resources.factory.s3.Bucket

In [4]:
# List all the files inside the bucket

for page in ds_raw.objects.pages():
    for obj in page:
        print(obj.key)

df_ds_tx.csv
df_ds_tx_backup.csv
ds_tx_indeed_02052021.csv
ds_tx_indeed_02102021.csv
ds_tx_indeed_02112021.csv


In [5]:
# Download file
database = env_Shi.database
s3.Bucket('dsrawjobpostings').download_file('df_ds_tx_backup.csv', 
                                            f"{database}df_ds_tx_aws.csv")

**Notes**: The file is successfully downloaded and saved in the customized location. 

### Read JSON Files from the Local Database

In [5]:
# Define a function to create the initials of the job title

def job_title_initials(job_title):
    '''
    This function accepts the job title in a string format (all lower case) and 
    returns the initials of the job titles.
    '''
    match = re.findall(r'([a-z])\w+', job_title)
    initials = ''.join(match)
    return initials

In [11]:
# Test function `job_title_initials`
job_title_initials('data scientist')

'ds'

In [6]:
# Define a function to load JSON file of job postings

def read_job_postings_json(job_title):
    '''
    This function reads the JSON file of prepared job postings into a pandas dataframe 
    based on a job title and set the date as the index.
    '''
    # Load the file path of the local database
    database = env_Shi.database
    # Create the file name
    initials = job_title_initials(job_title)
    file_name = 'df_' + initials + '_tx_prepared_backup.json'
    # Read the JSON file into a pandas dataframe
    df = pd.read_json(f'{database}{file_name}')
    # Print the numbr of job posts
    print("Number of Job Postings: ", df.shape[0])
    # Convert the string date to datetime
    df.date = pd.to_datetime(df.date)
    # Set the date as the index and sort the dataframe
    df = df.set_index('date').sort_index(ascending=False)
    return df

In [79]:
# Load the job postings of data scientist position in TX

df_ds_tx = read_job_postings_json('data scientist')
df_ds_tx.info()

Number of Job Postings:  2136
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2136 entries, 2021-02-21 to 2020-12-22
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title            2136 non-null   object 
 1   company          2136 non-null   object 
 2   company_rating   2136 non-null   float64
 3   job_link         2136 non-null   object 
 4   job_description  2136 non-null   object 
 5   city             2136 non-null   object 
 6   state            2136 non-null   object 
 7   zipcode          2136 non-null   int64  
 8   clean            2136 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 166.9+ KB


In [80]:
# Print the first two rows of the dataframe
df_ds_tx.head(2)

,title,company,company_rating,job_link,job_description,city,state,zipcode,clean
date,,,,,,,,,
2021-02-21,Vice President of Decision Science,CRESCENT BANK & TRUST,3.3,https://www.indeed.com/cmp/Crescent-Bank,error,Carrollton,TX,75006,error
2021-02-21,"Data Scientist, Analytics - Ads Delivery Product",Facebook,4.2,https://www.indeed.com/rc/clk?jk=3ec2fc58833a3...,The Ads Delivery Product team leverages state-...,Austin,TX,0,ad delivery product team leverage stateofthear...


In [45]:
df = pd.DataFrame({'name' : ['User 1', 'User 2', 'User 3']})
df

,name
0,User 1
1,User 2
2,User 3


In [50]:
# Store the dataframe as a aql

engine = create_engine('sqlite://', echo=False)
df.to_sql('users', con=engine)

2021-02-21 09:00:33,780 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-02-21 09:00:33,781 INFO sqlalchemy.engine.base.Engine ()
2021-02-21 09:00:33,783 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-02-21 09:00:33,783 INFO sqlalchemy.engine.base.Engine ()
2021-02-21 09:00:33,785 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("users")
2021-02-21 09:00:33,785 INFO sqlalchemy.engine.base.Engine ()
2021-02-21 09:00:33,786 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("users")
2021-02-21 09:00:33,787 INFO sqlalchemy.engine.base.Engine ()
2021-02-21 09:00:33,789 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	"index" BIGINT, 
	name TEXT
)


2021-02-21 09:00:33,789 INFO sqlalchemy.engine.base.Engine ()
2021-02-21 09:00:33,790 INFO sqlalchemy.engine.base.Engine COMMIT
2021-02-21 09:00:33,791 INFO sqlalchemy.engine.base.Engine CREATE INDEX ix_users_index ON 

In [49]:
engine.execute("SELECT * FROM users").fetchall()

[(0, 'User 1'), (1, 'User 2'), (2, 'User 3')]

In [35]:
# Print the top 10 companies by the number of posts
df_ds_tx.company.value_counts().head(10)

Cognizant Technology Solutions    63
Dell Technologies                 46
Deloitte                          44
Facebook                          44
USAA                              41
StataCorp                         32
JPMorgan Chase Bank, N.A.         31
Pearson                           30
Accenture                         27
Apple                             27
Name: company, dtype: int64

In [8]:
# Print the top 10 cities by the number of posts
df_ds_tx.city.value_counts().head(10)

Austin             593
Dallas             346
Houston            261
San Antonio        162
Plano              161
Irving             106
0                   97
Fort Worth          58
Round Rock          40
College Station     34
Name: city, dtype: int64

In [9]:
# Sanity check: the dataframe has datetime index
df_ds_tx.resample("W").title.count()

date
2020-12-27    392
2021-01-03    136
2021-01-10    212
2021-01-17    187
2021-01-24    345
2021-01-31    288
2021-02-07    245
2021-02-14    202
2021-02-21     60
Freq: W-SUN, Name: title, dtype: int64

### Find the Company Address

In [13]:
df_ds_tx.job_link[1]

'https://www.indeed.com/rc/clk?jk=e156912ab44a68af&fccid=72af0536a73f5103&vjs=3'

### Extract Job Requirements by Regular Expression

In [ ]:
# Take a random job link

job_url = df_ds.job_link.sample(1, random_state=1)[0]
job_url

In [ ]:
# Make the rquest

response = requests.get(job_url)
response.status_code

In [ ]:
# Make a soup to hold the response content
soup = BeautifulSoup(response.content, 'html.parser')
soup.title.string

In [ ]:
soup.style

In [ ]:
print(soup.prettify())

In [ ]:
# Create 'words' variable
words = [re.sub(r'([^a-z0-9\s]|\s.\s)', '', doc).split() for doc in df_ds_tx.clean]

# Add 'words' column to dataframe
# Column will contain lists of separated words in each repo
df_ds_tx = pd.concat([df_ds_tx, pd.DataFrame({'words': words})], axis=1)

df_ds_tx.head(2)

## Frequency Analysis of Mono-, Bi-, and Tri-grams

### Create a list of all the words appear in the job descriptions

In [21]:
# Define the function to create the words that appear in the job descriptions

def words_variables_v1(df):
    '''
    This function accepts the dataframe with cleaned job description 
    and return a dictionary in which the values are the words that 
    appear in the job description. 
    '''
    # Create the words that appear all the job descritipons
    all_words = ' '.join(df.clean)
    # Create a dictionary to hold the variable all_words
    d_words = {'frequency': all_words}
    return d_words

In [30]:
# Test the helper function: words_variables_v1
dic = words_variables_v1(df_ds_tx)

# Print out the keys
print(dic.keys())

# Print the first 100 characters of the value
dic['frequency'][:100]

dict_keys(['frequency'])


'opportunity develop team mentor manager develop innovative data science solution utilize machine lea'

In [ ]:
# Upgrade the function `words_variables_v1`

def words_variables_v2(df, companies):
    '''
    This function accepts the dataframe containing cleaned job description and 
    a list of company names and return a dictionary in which the values are the words 
    that appear in the job description. 
    '''
    # Create the words that appear all the job descritipons
    all_words = ' '.join(df.clean)
    # Create a dictionary to hold the variable all_words
    d_words = {'all': all_words}
    # For loop the companies and create the words that appear in their job descriptions
    for company in companies:
        mask = (df.company == company)
        s_company = df[mask].clean
        words = ' '.join(s_company)
        d_words[company] = words
    return d_words

In [ ]:
# Test the helper function: words_variables_v2

companies = ['Apple']
dic_v2 = words_variables_v2(df_ds_tx, companies)

# Print out the keys
print(dic_v2.keys())

# Print the first 100 characters of the value of `Apple`
dic_v2['Apple'][:400]

### Monogram Analysis

In [ ]:
# Define a function to compute the word frequency in the job description

def word_frequency_v1(d_words):
    '''
    This function accept the dictionary created by function words_variables_v1
    and return the word frequency in the job description. 
    '''
    # Create a dataframe to hold the word frequency
    word_counts = pd.DataFrame()
    # Compute the words frequency
    freq = pd.Series(d_words['frequency'].split()).value_counts()
    word_counts = pd.concat([word_counts, freq], axis=1, sort=True)
    word_counts.columns = d_words.keys()
    word_counts.sort_values(by='frequency', ascending=False, inplace=True)
    return word_counts

In [ ]:
# Upgrade `word_frequency_v1`

def word_frequency_v2(d_words):
    '''
    This function accept the dictionary created by function words_variables_v2
    and return the word frequency in the job description. 
    '''
    # Read the company names from the dictionary
    companies = d_words.keys()
    # Create a dataframe to hold the word frequency
    word_counts = pd.DataFrame()
    # For loop through the companies and generate the word frequency in their job descriptions
    for company in companies:
        freq = pd.Series(d_words[company].split()).value_counts()
        word_counts = pd.concat([word_counts, freq], axis=1, sort=True)
    word_counts.columns = companies
    word_counts = word_counts.fillna(0).apply(lambda s: s.astype(int))
    word_counts.sort_values(by='all', ascending=False, inplace=True)
    return word_counts

### Bigram Analysis

In [ ]:
# # Added 'Bigram' column to dataframe
# df_ds_tx['bigrams'] = [list(nltk.ngrams(wordlist, 2)) for wordlist in df_ds_tx.words]
# df_ds_tx.head(2)

In [ ]:
def bigrams_frequency_v1(d_words):
    '''
    This function accept the dictionary created by function words_variables_v1
    and return the word frequency in the job description. 
    '''
    # Create a dataframe to hold the word frequency
    word_counts = pd.DataFrame()
    # Compute the words frequency
    freq = pd.Series(list(nltk.ngrams(d_words['frequency'].split(), 2))).value_counts()
    # Add the `freq` seires to `word_counts` dataframe
    word_counts = pd.concat([word_counts, freq], axis=1, sort=True)
    # Rename the coumns
    word_counts.columns = d_words.keys()
    # Sort the dataframe by the values in column `frequency`
    word_counts.sort_values(by='frequency', ascending=False, inplace=True)
    return word_counts

In [ ]:
# Define a function to compute the bigrams frequency in the job description

def bigrams_frequency_v2(d_words):
    '''
    This function accept the dictionary created by function words_variables_v2
    and return the bigrams frequency in the job description. 
    '''
    # Read the company names from the dictionary
    companies = d_words.keys()
    # Create a dataframe to hold the word frequency
    bigrams_counts = pd.DataFrame()
    # For loop through the companies and generate the word frequency in their job descriptions
    for company in companies:
        freq = pd.Series(list(nltk.ngrams(d_words[company].split(), 2))).value_counts()
        bigrams_counts = pd.concat([bigrams_counts, freq], axis=1, sort=True)
    bigrams_counts.columns = companies
    bigrams_counts = bigrams_counts.fillna(0).apply(lambda s: s.astype(int))
    bigrams_counts.sort_values(by='all', ascending=False, inplace=True)
    return bigrams_counts

### Trigram Analysis

In [ ]:
def trigrams_frequency_v1(d_words):
    '''
    This function accept the dictionary created by function words_variables_v1
    and return the word frequency in the job description. 
    '''
    # Create a dataframe to hold the word frequency
    word_counts = pd.DataFrame()
    # Compute the words frequency
    freq = pd.Series(list(nltk.ngrams(d_words['frequency'].split(), 3))).value_counts()
    # Add the `freq` seires to `word_counts` dataframe
    word_counts = pd.concat([word_counts, freq], axis=1, sort=True)
    # Rename the coumns
    word_counts.columns = d_words.keys()
    # Sort the dataframe by the values in column `frequency`
    word_counts.sort_values(by='frequency', ascending=False, inplace=True)
    return word_counts

In [ ]:
# Define a function to compute the trigrams frequency in the job description

def trigrams_frequency_v2(d_words):
    '''
    This function accept the dictionary created by function words_variables_v2
    and return the trigrams frequency in the job description. 
    '''
    # Read the company names from the dictionary
    companies = d_words.keys()
    # Create a dataframe to hold the word frequency
    trigrams_counts = pd.DataFrame()
    # For loop through the companies and generate the word frequency in their job descriptions
    for company in companies:
        freq = pd.Series(list(nltk.ngrams(d_words[company].split(), 3))).value_counts()
        trigrams_counts = pd.concat([trigrams_counts, freq], axis=1, sort=True)
    trigrams_counts.columns = companies
    trigrams_counts = trigrams_counts.fillna(0).apply(lambda s: s.astype(int))
    trigrams_counts.sort_values(by='all', ascending=False, inplace=True)
    return trigrams_counts

### Combine Mono-, Bi- and Trigrams

#### Method 1: Simple concatenation

#### Method 2:  Use nltk.util.everygrams

In [31]:
# Define a function to compute the frequence of the mono-, bi-, and tri-grams of the job description

def everygram_frequency_v1(d_words, max_len=3):
    '''
    This function accetps the dictionary produced by the function `words_variables_v1` and 
    return mono-, bi-, and tri-grams along with their frequency. 
    '''
    # Generate mono-, bi-, and tri-grams
    grams = nltk.everygrams(d_words['frequency'].split(), max_len=max_len) # dtype of grams: <class 'genertor'>
    # Convert to a list of tuples
    grams = list(grams)
    # Create an empty list to hold mono-, bi-, and tri-grams
    everygram = []
    # For loop the list of tuples and convert the grams to strings
    for gram in grams:
        str_gram = gram[0]
        for i in gram[1:]:
            str_gram = str_gram + ' ' + i
        everygram.append(str_gram)
    # Compute the frequency of the everygrams
    everygram = pd.Series(everygram).value_counts()
    return everygram

In [32]:
# Test the function above

everygram = everygram_frequency_v1(dic)
everygram.head()

data          16501
experience     9129
business       5749
team           5112
work           4516
dtype: int64

In [36]:
# Save the ds_grams as json file
everygram.to_json(f"{database}ds_grams.json", orient='index')

In [39]:
# Uploade to AWS s3 bucket

# Create the s3 resource object
s3 = boto3.resource('s3')

# Read the location of the database
database = env_Shi.database

# Upload df_ds_tx_backup.csv file
s3.Bucket('additionaljobinfo').upload_file(f"{database}ds_grams.json", "ds_grams.json")

### Compute Top 5 Skills in a Predifined Library

In [37]:
# Define a function to pick up the top k skills of a data scientict from a skillset library

def top_skills_ds_v1(k, library):
    '''
    This function accepts a positive integer k and a skillset library and 
    returns a dataframe containing the top k skills needed for data scientist positions.
    '''
    # Import the file path
    database = env_Shi.database
    # Load the prepared dataframe with job search results
    df = pd.read_json(f"{database}df_ds_tx_prepared_backup.json")
    # Create a string of all words that appear in the job description
    dic = words_variables_v1(df)
    # Compute the words frequency
    everygram_frequency = everygram_frequency_v1(dic)
    # Create a empty dataframe to hold the rank of the skills
    df_skills = pd.DataFrame()
    # For loop through the library to find out the frequency of the skills mentioned in the job description
    for skill in library:
        mask = (everygram_frequency.index == skill)
        df =  everygram_frequency[mask]
        df_skills = pd.concat([df_skills, df])
    df_skills.columns = dic.keys()
    df_skills.sort_values(by='frequency', ascending=False, inplace=True)
    return df_skills.head(k)

In [48]:
# Create a tech library
tech_library = ['python', 'sql', 'pandas','numpy','matplotlib','scikit learn','spark','hadoop',
           'aws','amazon web services','azure','microsoft word','microsoft excel','excel',
           'tableau','tensor flow','pytorch','hive', 'impala', 'matlab','etl',
           'statistics','exploration', 'extraction', 'data wrangling','math',
           'machine learning','data visualization','java','js',
           'javascript','scala','r','c','c++','power bi','dashboard','linear algebra',
           'calculus','neural networks','eda','big data','frameworks','database management',
           'testing hypotheses','probability','data mining','perl','nosql','saas','git',
           'github','natural language processing','nlp','deep learning','agile','kanban',
           'project management','julia','devops','google cloud','pytorch','computer vision']

# Print the number of skills in the library
print("Number of skills in tech skill library: ", len(tech_library))

# Test function: top_skills_ds_v1
df_test = top_skills_ds_v1(20, tech_library)
df_test

Number of skills in tech skill library:  63


,frequency
machine learning,2521.0
python,1329.0
sql,1012.0
r,760.0
aws,689.0
big data,622.0
spark,569.0
hadoop,539.0
c,442.0
java,439.0


In [46]:
# Pick up the top 20 soft skills
soft_library = ['critical thinking','communication','problem solving','teamwork','ethics','business acumen',
           'interpersonal skills','curiosity','storytelling','adaptability','team player','collaboration',
                'time management','leadership','domain knowledge','creativity','decision making',
           'verbal communication','written communication']

# Print the number of skills in the library
print("Number of skills in soft skill library: ", len(soft_library))

top_soft_skills = top_skills_ds_v1(20, soft_library)
top_soft_skills

Number of skills in soft skill library:  20


,frequency
learning,3576.0
communication,1122.0
leadership,707.0
collaboration,348.0
problem solving,244.0
written communication,199.0
decision making,146.0
verbal communication,109.0
curiosity,93.0
creativity,91.0


### Skills Match Job Search

In [24]:
# Define a function to find the job position that match the skills of a applicant

def skills_match_search(job_title, skills):
    '''
    '''  
    # Create the initials of the job_title
    initials = job_title_initials(job_title)
    # Load the file path
    database = env_Shi.database
    # Create the file name
    file_name = 'df_' + initials + '_tx_prepared_backup.json'
    # Load the job postings file
    df = pd.read_json(f'{database}{file_name}')
    # Create a list variable to hold the boolean values
    mask = []
    # For loop 
    for clean in df.clean:
        if all(skill in clean for skill in skills):
            mask.append(True)
        else:
            mask.append(False)
    df_match = df[mask]
    # Drop redudant columns
    cols = ['date', 'zipcode', 'clean', 'tokenized', 'stemmed', 'lemmatized']
    df_match.drop(columns=cols, inplace=True)
    print("Number of the Matched Companies: ", df_match.shape[0])
    return df_match

In [25]:
skills = ['python', 'sql', 'tableau', 'aws']

df = skills_match_search('data scientist', skills)
df.shape

Number of the Matched Companies:  72


(72, 7)

In [28]:
df.sample(2)

,title,company,company_rating,job_link,job_description,city,state
464,Cloud Data Engineer,Spectral MD,0.0,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,"Company OverviewSpectral MD, Inc. is a medical...",Dallas,TX
370,Cloud Data Engineer\nnew,Spectral MD,0.0,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,"Company OverviewSpectral MD, Inc. is a medical...",Dallas,TX


In [41]:
# Create the masks for different skills

mask_python = df_ds_tx.clean.str.contains('python')
mask_sql = df_ds_tx.clean.str.contains('sql')
mask_ml = df_ds_tx.clean.str.contains('machine learning')
mask_tableau = df_ds_tx.clean.str.contains('tableau')
mask_aws = df_ds_tx.clean.str.contains('aws')

mask = mask_python & mask_sql & mask_tableau & mask_aws

In [42]:
# How many companies need all three skills: python, sql and tableau
mask.sum()

72

In [19]:
df_ds_tx[mask].head(1)

,title,company,company_rating,job_link,job_description,city,state,zipcode,clean,tokenized,stemmed,lemmatized
date,,,,,,,,,,,,
2021-02-19,"Analyst, Data Science - Product Analytics",Expedia Group,3.9,https://www.indeed.com/rc/clk?jk=e447fed7ec145...,This is a great opportunity to join Vrbo’s glo...,Austin,TX,78758,great opportunity join vrbos global analytics ...,this is a great opportunity to join vrbos glob...,thi is a great opportun to join vrbo global an...,this is a great opportunity to join vrbos glob...


In [ ]:
df_ds_tx.clean[0][:100]

### Geospatial Analysis

In [31]:
x for x in range(5)

SyntaxError: invalid syntax (<ipython-input-31-ff77852d28dc>, line 1)